In [ ]:
!pip install tiktoken==0.8.0 wandb==0.19.6 weave==0.51.33 docker==7.1.0 openai==1.61.1

In [ ]:
import os
import asyncio
import docker
from openai import OpenAI
import wandb
import weave

In [ ]:
openai_client = OpenAI()

In [ ]:
async def create_container(docker_client):
    container_name = "nbtest_wandb"
    try:
        container = docker_client.containers.get(container_name)
    except docker.errors.NotFound:
        # Ensure network exists
        network_name = "wandb_network"
        try:
            network = docker_client.networks.get(network_name)
        except docker.errors.NotFound:
            network = docker_client.networks.create(network_name)
    
        # Create container
        container = docker_client.containers.run(
            image="code-runner-client",
            name=container_name,
            shm_size="512mb",
            restart_policy={"Name": "unless-stopped"},
            network=network_name,
            detach=True,
            command=["/bin/sh", "-c", "sleep infinity"],
        )
        await asyncio.sleep(3)
    
    return container

In [ ]:
async def run_container_command(container):
    exec_result = container.exec_run(
        cmd=["python", "-c", "print('Hello, World!')"], stdout=True, stderr=True
    )
    return exec_result.output.decode("utf-8")

In [ ]:
async def run_openai_command():
    oai_response = openai_client.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=[
            {
                "role": "system",
                "content": "You are a helpful assistant.",
            },
            {"role": "user", "content": "Please help me createt he following task..."},
        ],
        temperature=0.7,
        max_tokens=1500
    )
    completion_response = oai_response.choices[0].message.content
    return completion_response

In [ ]:
async def main():
    task = ""
    docker_client = docker.from_env()
    container = await create_container(docker_client)
    completion = await run_openai_command()        
    result = await run_container_command(container)
    output = f"""
Command output:
{result.strip()}
---
OpenAI output:
{completion.strip()}
    """.strip()
    print(output)
await main()